In [ ]:
include("polynomials.jl")
include("quotientrings.jl")

Base.show(io::IO, p::Polynomial{Quot{T, Q}}) where {T <: Number, Q <: Val} = 
print(io, join(reverse!(["$(p[i].a) 𝕩^$i" for i ∈ 0:p.degree]), " + ") * "    mod $(getval(Q))")

# An application of Shamir Secret Sharing to computing sums

Suppose $n$ people with secret numbers $s_1, s_2, \ldots, s_n$ want to compute the sum of their secret numbers without revealing the individual numbers.


This can be accomplished through a modification of the Shamir Secret Sharing (SSS) protocol. Another variant of this approach yields a method for computing the product of the secret numbers, and consequently, any polynomial function of the variables $s_1, s_2, \ldots, s_n$.

The assumptions are:
- There is a secret communication channel between each pair of players
- A majority of the players are honest
- No player deviates from the protocol

In [ ]:
mutable struct Player{T} <: Any where {T <: Number}
    name::Symbol
    number::Int
    
    secret::T
    secretpoly::Polynomial{T}
    shares::Vector{T}
    
    computedvalue::T
end

In [ ]:
Base.show(io::IO, p::Player) = print(io, "Player $(p.number): $(p.name)")

In [ ]:
function randompolynomial(D; degree) 
    Polynomial([D; rand(typeof(D), degree)])
end

In [ ]:
Player(name::Symbol, number::Int; num_players = N, threshold = t, 𝔽ield = 𝔽) = begin
    Player(name, number, rand(𝔽ield), Polynomial([zero(𝔽ield)]), zeros(𝔽ield, num_players), zero(𝔽ield))
end

**The set-up:**
A group of $n$ people, with secret numbers $s_1, s_2, \ldots, s_n$ respectively, want to securely compute $(t, n)$ secret shares for the sum of their numbers.

**Implementation:**
Consider a set-up with $5$ people performing a $(3, 5)$ secret sharing with secret numbers drawn from the prime field $\mathbb{Z}_{17}$.

In [ ]:
𝔽 = Quot{Int, Val{17}};  #the field
N, t = 5, 2;  #the number of players and the threshold

In [ ]:
Anand = Player(:Anand, 1)
Anirudh = Player(:Anirudh, 2)
Hrishikesh = Player(:Hrishikesh, 3)
Kapil = Player(:Kapil, 4)
Srijay = Player(:Srijay, 5)

players = (Anand, Anirudh, Hrishikesh, Kapil, Srijay)

#### Step 1: Input sharing

Every player executes the $(t, n)$ Shamir Secret Sharing protocol with their secrets.

This differs from the regular protocol in two important ways:
- Each player receives a share of their own secret
- Instead of evaluating the secret polynomials at random inputs, the $i$th player is always given the share corresponding to the input $i$

In [ ]:
for p ∈ players;        p.secretpoly = randompolynomial(p.secret; degree = t);     end

In [ ]:
for p ∈ players, q ∈ players;     q.shares[p.number] = p.secretpoly(q.number);     end

#### Step 2: Sum evaluation

Each player then computes the sum of the shares they have received from the other players. The shares act like proxies for the values of the inputs.

For the $i$th player, the sum of the shares they have received is

$$
\sum_{p \in players} p.secretpoly(i)
$$

Consider a new polynomial

$$
Q(x) = \sum_{p \in players} p.secretpoly(x)
$$

This is a degree $t$ polynomial with constant coefficient equal to 
$$Q(0) = \sum_{p \in players} p.secretpoly(0) = \sum_{p \in players} p.secret$$
Although the players have no way of computing $Q(x)$ without revealing their secret polynomials, they can individually compute the values of the shares $Q(i)$ by summing the shares received from the other players (as mentioned above).

In [34]:
for p ∈ players;     p.computedvalue = sum(p.shares);  end

#### Step 3: Output construction

Now that the players have all computed their individual shares of the $t$-degree polynomial $Q(x)$, they can find the secret $Q(0)$ - which is the sum of their secret numbers - by combining any $t + 1$ shares and finding the interpolating polynomial.

In [35]:
function interpolate(xs, ys::Vector{T}) where {T}
    𝕩 = Polynomial([zero(T), one(T)])
    
    #L(j) = prod([(𝕩 - Polynomial([xₖ])) for (k, xₖ) ∈ enumerate(xs) if k != j]) * 
    #Polynomial([inv(prod([(xs[j] - xₖ) for (k, xₖ) ∈ enumerate(xs) if k != j]))])
    
    L(j) = prod( [
            (𝕩 - Polynomial([xₖ])) * Polynomial([inv(xs[j] - xₖ)])
            for (k, xₖ) ∈ enumerate(xs) if k != j
            ] )
    
    sum([Polynomial([yⱼ]) * L(j) for (j, yⱼ) ∈ enumerate(ys)])
end       

interpolate(ps::Vector{Player{T}}) where {T <: Any} = interpolate(getfield.(ps, :number) .+ zero(T), getfield.(ps, :computedvalue))   

interpolate (generic function with 2 methods)

In [36]:
interpolate([Anirudh, Kapil, Srijay])

12 𝕩^2 + 13 𝕩^1 + 0 𝕩^0    mod 17

In [37]:
sum((getfield.(players, :secret)) .* [0, 5, 3, 1, 4])

7  mod  17